# VERY IMPORTANT NOTE
# You have to connect to TPUv2 or this will NOT WORK.

Go to runtime->change runtime type <br>
Then select TPUv2 <br>
You will need Colab Pro, but sometimes google allows anyone to connect to their TPUv2 -- give it a shot

# Set-up

In [ ]:
# Packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
import os
from IPython.display import clear_output

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

# Keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
#from keras.layers.embeddings import Embedding
from keras.metrics import AUC

# Tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import random

# Important Params
n_layers = 2
patch_length = 18

In [ ]:
## SEEDS

# Hard Code Random Seeds.
r1 = 0
r2 = 1

# Set Random Seed
random.seed(r1)
tf.random.set_seed(r2)

In [ ]:
#@title Connect to TPU
print("TensorFlow version:", tf.__version__)

# Connect to the TPU cluster or fall back to CPU/GPU
try:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # Tries to connect to the TPU
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.TPUStrategy(resolver)
  devices = tf.config.list_logical_devices('TPU')
  print('TPU devices:', devices)
except ValueError:
  print("Could not connect to TPU; using CPU/GPU strategy instead.")
  strategy = tf.distribute.get_strategy()

# Example computation using the strategy
with strategy.scope():
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

  @tf.function
  def matmul_fn(x, y):
    return tf.matmul(x, y)

  z = strategy.run(matmul_fn, args=(a, b))

print(z)

TensorFlow version: 2.15.0
TPU devices: [LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]
PerReplica:{
  0: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  1: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  2: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  3: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  4: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  5: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  6: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  7: tf.

# Download Data from our Dropbox

To make a dropbox link a downloadable link just change that dl=0 at the end of the link into dl=1

In [ ]:
import requests

# Downloads any file from the internet function
def download_file(url, filename):
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)

In [ ]:
# Download X_test and y_test file from the internet

####################
# Download y_test
####################

# URL for direct download from our Google Drive
url = 'https://www.dropbox.com/scl/fi/n9cpbawvavv3d90gg6fsw/y_test.npy?rlkey=44ltf1u0k4wltfsgryt3z5z4e&st=5vuj5amo&dl=1'
filename = 'y_test.npy'

# Download the file
download_file(url, filename)

# Load the .npy file into a numpy array
y_test = np.load(filename)

####################
# Download X_test
####################

# URL for direct download from our Google Drive
url = 'https://www.dropbox.com/scl/fi/nho45032np92afgpmn9o0/X_test.npy?rlkey=qyb72seovnifmlx2g2fhdyooh&st=bu1cqp45&dl=1'
filename = 'X_test.npy'

# Download the file
download_file(url, filename)

# # Load the .npy file into a numpy array
X_test = np.load(filename)

####################
# Download Model Weights
####################

# URL for direct download from our Google Drive
url = 'https://www.dropbox.com/scl/fi/vnc6xgc6a29gxkkcn6o3l/my_model_weights.h5?rlkey=m2mqep3ltjezfihktidw21amf&st=tmdvfm22&dl=1'
model_weights_path = 'my_model_weights.h5'

# Download the file
download_file(url, model_weights_path)



https://www.dropbox.com/scl/fi/vnc6xgc6a29gxkkcn6o3l/my_model_weights.h5?rlkey=m2mqep3ltjezfihktidw21amf&st=tmdvfm22&dl=0

# Load Pretrained Model

### Outline Model Architecture

In [ ]:
# Define the Transformer block
def TransformerBlock(embed_dim, num_heads, ff_dim, rate=0.1):
    input_layer = tf.keras.layers.Input(shape=(None, embed_dim))
    attention_layer = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    attention_output, attention_weights = attention_layer(input_layer, input_layer, return_attention_scores=True)
    attention_output = tf.keras.layers.Dropout(rate)(attention_output)
    out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(input_layer + attention_output)
    ff_output = tf.keras.layers.Dense(ff_dim, activation="relu")(out1)
    ff_output = tf.keras.layers.Dense(embed_dim)(ff_output)
    ff_output = tf.keras.layers.Dropout(rate)(ff_output)
    final_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(out1 + ff_output)
    return tf.keras.Model(inputs=input_layer, outputs=[final_output, attention_weights])

# Define the main model
def create_model(input_size=10080, patch_size=patch_length, embed_dim=96, num_heads=12, ff_dim=256, num_layers=n_layers, rate=0.1, training=True):
    num_patches = input_size // patch_size
    inputs = tf.keras.layers.Input(shape=(input_size,))

    reshaped = tf.keras.layers.Reshape((num_patches, patch_size))(inputs)
    patch_embeddings = tf.keras.layers.Dense(embed_dim)(reshaped)

    positional_emb = tf.keras.layers.Embedding(input_dim=num_patches, output_dim=embed_dim)(tf.range(num_patches))
    x = patch_embeddings + positional_emb

    attention_weights = []

    for _ in range(num_layers):
        x, weights = TransformerBlock(embed_dim, num_heads, ff_dim, rate)(x)
        if not training:
            attention_weights.append(weights)

    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dropout(rate)(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    if training:
        return tf.keras.Model(inputs=inputs, outputs=outputs)
    else:
        return tf.keras.Model(inputs=inputs, outputs=[outputs] + attention_weights)


Create the structure of the model and then change the weights.

In [ ]:
# Compile the model -----
with strategy.scope():
  model = create_model(training=True)
  model.compile(
    # Metrics
    loss= tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics= tf.keras.metrics.AUC(name='auc'),
    # Optimizer
    optimizer= tf.keras.optimizers.Adam(
      learning_rate=0.00001,
      beta_1=0.9,
      beta_2=0.999,
      epsilon=1e-07,
      amsgrad=False
))

model.load_weights(model_weights_path)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10080)]           0         
                                                                 
 reshape (Reshape)           (None, 560, 18)           0         
                                                                 
 dense (Dense)               (None, 560, 96)           1824      
                                                                 
 tf.__operators__.add (TFOp  (None, 560, 96)           0         
 Lambda)                                                         
                                                                 
 model (Functional)          [(None, None, 96),        495808    
                              (None, 12, None, None)             
                             ]                                   
                                                           

### Results will be displayed upon running the kernel below

**Note:** You do need to use TPUv2 to get similar results. The way that the model computes changes slightly if are to use CPU, and may change the loss and auc scores.

In [ ]:
model.evaluate(X_test, y_test, verbose=1)

92/92 [==============================] - 6s 31ms/step - loss: 0.7523 - auc: 0.7218


[0.7522586584091187, 0.7217706441879272]